In [7]:
import numpy as np
import random as rd
import scipy.stats as sc

lberm = 35
l = 20
n = 10000000
B = 37.5
xexit = 18.75 + lberm
phi = 37
d70m = 2.08e-4
v = 1.33e-6
g = 9.81
yw = 10
ys = 26.50

# Water level
data = np.loadtxt('Cumulative_density_function_water_level.txt', delimiter=' ', unpack=True)

def lognorm_params(mode, stddev):
    p = np.poly1d([1, -1, 0, 0, -(stddev/mode)**2])
    r = p.roots
    sol = r[(r.imag == 0) & (r.real > 0)].real
    shape = np.sqrt(np.log(sol))
    scale = mode * sol
    return shape, scale

# Start
Pf = []
for i in range(l):
    # Generate Stochastic Lognormal
    sigma, scale = lognorm_params(25, 0.5)
    D = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(4.5, 0.5)
    d = sc.lognorm.rvs(sigma, 0, scale, size=n) 
    sigma, scale = lognorm_params(7.52e-4, 0.5 * 7.52e-4)
    k = sc.lognorm.rvs(sigma, 0, scale, size=n)
    sigma, scale = lognorm_params(1.00e-6, 0.5 * 1.00e-6)
    kh = sc.lognorm.rvs(sigma, 0, scale, size=n) 
    sigma, scale = lognorm_params(20, 2)
    Lf = sc.lognorm.rvs(sigma, 0, scale, size=n)  
    sigma, scale = lognorm_params(2.8e-4, 0.12*2.8e-4)
    d70 = sc.lognorm.rvs(sigma, 0, scale, size=n)
    
    # Generate Stochastic Normal
    hp = sc.norm.rvs(3.50, 0.1, n)
    mp = sc.norm.rvs(1.0, 0.12, n)
    
    # Generate water level
    h = []
    for q in range(n):
        w = rd.random()
        h.append(np.interp(w, data[1], data[0]))
    
    # Calc
    L = Lf + B + lberm
    lambda_h = ((k * D * d) / kh) ** 0.5
    lambda_ = (lambda_h / (Lf + B + lambda_h)) * np.exp((0.5 * B - xexit) / lambda_h)
    F1 = 0.25 * ((ys / yw) - 1) * np.tan(np.radians(phi))
    F2 = (d70m / (((v * k * L) / g) ** (1/3))) * (d70 / d70m) ** 0.4
    F3 = 0.91 * (D / L) ** ((0.28 / ((D / L)**2.8 - 1)) + 0.04)
    H = h - hp - 0.3 * d
    Hcp = F1 * F2 * F3 * L
    Z = mp * Hcp - H
    
    fail = 0
    for q in range(n):
        if(Z[q] < 0):
            fail += 1
    
    if(fail > 0):
        Pf.append(fail / n)
    else:
        Pf.append(0)
        
    print(i, "Pf:", Pf[i])
    
print("\nTOTAL Pf:", round(np.sum(Pf) / l, 5))
print(np.sum(Pf))

0 Pf: 1.14e-05
1 Pf: 1.35e-05
2 Pf: 9e-06
3 Pf: 1.23e-05
4 Pf: 1.2e-05
5 Pf: 1.23e-05
6 Pf: 1.24e-05
7 Pf: 1.4e-05
8 Pf: 1.04e-05
9 Pf: 1.25e-05
10 Pf: 1.2e-05
11 Pf: 1.42e-05
12 Pf: 1.02e-05
13 Pf: 1.11e-05
14 Pf: 1.15e-05
15 Pf: 1.14e-05
16 Pf: 1.3e-05
17 Pf: 1.08e-05
18 Pf: 1.12e-05
19 Pf: 1.3e-05

TOTAL Pf: 1e-05
0.0002382
